# Development

Agreement and policy: https://developer.twitter.com/en/developer-terms/agreement-and-policy

In [ ]:
import os
os.chdir('..')

import tweepy
import openai
import requests
import psycopg2
import pandas as pd

from authenticators import (
    TwitterAuthenticator, OpenAIAuthenticator, RDSAuthenticator
)

twitter = TwitterAuthenticator()
client = tweepy.Client(bearer_token=twitter.bearer_token)

gpt = OpenAIAuthenticator()
openai.api_key = gpt.api_key

rds = RDSAuthenticator()
connection = psycopg2.connect(
    host = rds.host,
    port = rds.port,
    database= rds.database,
    user = rds.user,
    password = rds.password
)
connection.autocommit = True
cursor = connection.cursor()

In [2]:
import os
os.chdir('..')

from modules import TwitterDataCleaner, get_tweets

query = 'sandra torres'
start_time = "2023-04-26T00:00:00Z" 
end_time = "2023-04-30T00:00:00Z" 
max_results = 10

tweets_call = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

data_cleaner = TwitterDataCleaner(tweets_call)
tweets, users = data_cleaner.run()

In [4]:
tweets.tail()

,tw_tweet,tw_usuario,tw_fecha,tw_texto,tw_sensitivo,tw_retweets,tw_replies,tw_likes,tw_quotes,tw_impresiones,tw_idioma
5,1652403078455328768,221932544,2023-04-29,"Durante su visita a Chichicastenango, Quiché, ...",False,94,103,181,4,18786,es
6,1652387604652781569,1095086472389111811,2023-04-29,¿Quitar el IVA a los medicamentos? 💵💊\n\nEsta ...,False,2,1,6,1,522,es
7,1652369298260623364,1561217740920193029,2023-04-29,quítenle el tiktok a sandra torres,False,0,0,1,0,19,es
8,1652358326955458560,1098537742349320193,2023-04-29,"🗣️ Na Renova, em Torres Novas, a deputada Sand...",False,0,1,5,0,146,pt
9,1652356666929606657,1420560045323038720,2023-04-29,"El antivoto no es para Sandra Torres, el antiv...",False,0,0,0,0,14,es


¿En qué necesito pensar?

- Conectarme a RDS
- Esquema para extraer datos uniformemente durante los últimos 7 días
- Establecer qué políticos tomar en cuenta. Scrape the list from here: https://es.wikipedia.org/wiki/Elecciones_generales_de_Guatemala_de_2023
- ¿Guardar los datos en una base de datos SQL local?


## Zero-shot feature extraction with GTP-3.5

In [ ]:
import re

def remove_emojis_and_links(text):
    # Remove URLs
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = url_pattern.sub('', text)

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub('', text)

    # Remove newline characters
    text = text.replace('\n', ' ')

    return text

In [ ]:
example = tweets.text[9]

tweet = remove_emojis_and_links(example)
tweet

In [ ]:
prompt = f"""
El siguiente es un tweet en español. Por favor, clasificalo de acuerdo a las siguientes categorías:

Valencia: [positivo, negativo, neutral]
Emoción: [felicidad, tristeza, enojo, miedo, sorpresa, disgusto, neutral]
Postura: [aprobación, desaprobación, esperanza, desilusión, indiferencia, condianza, desconfianza]
Tono: [agresivo, pasivo, asertivo, escéptico, irónico, humorístico, informativo, serio, inspiridor, otro]
Tema: [política, economía, deportes, entretenimiento, tecnología, ciencia, salud, educación, religión, cultura, medio ambiente, otro]

Además, evalúalo utilizando una escala continua con rango de 0 a 1 en las siguientes dimensiones:

Amabilidad: [0.0 - 1.0]
Legibilidad: [0.0 - 1.0]
Controversialidad: [0.0 - 1.0]
Informatividad: [0.0 - 1.0]
Espectro político: [0.0 - 1.0] (0 = izquierda, 1 = derecha)

Formatea tu respuesta como un diccionario de Python con las siguientes llaves:

[valencia, emocion, postura, tono, tema, amabilidad, legibilidad, controversialidad, informatividad]

Tweet: '''{tweet}'''
"""

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [ ]:
test = get_completion(prompt, temperature=0)
test = test.replace('\n', ' ')

In [ ]:
import json
dict = json.loads(test)
dict

In [ ]:
pd.DataFrame(dict, index=[0])

## Store them in a RDS instance

In [ ]:
cursor.close()
connection.close()

In [ ]:
# Create a sample DataFrame
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

# Upload the DataFrame to your PostgreSQL instance
# Replace "your_table_name" with the desired table name
df.to_sql('your_table_name', engine, if_exists='replace', index=False)


## Example usage

In [ ]:
query = 'sandra torres'
start_time = "2023-04-25T00:00:00Z" 
end_time = "2023-04-28T00:00:00Z" 
max_results = 10

tweets_call = get_tweets(
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=max_results,
)

data_cleaner = TwitterDataCleaner(tweets_call)
tweets, users = data_cleaner.clean()

In [ ]:
tweets.head()

In [ ]:
prefix = "us_"

users_renamed = (
    users.rename(columns={
        "user_id": f"{prefix}usuario",
        "user_username": f"{prefix}handle",
        "user_name": f"{prefix}nombre",
        "user_location": f"{prefix}ubicacion",
        "user_created_at": f"{prefix}fecha_creacion",
        "user_description": f"{prefix}descripcion",
        "user_profile_image_url": f"{prefix}imagen",
        "user_verified": f"{prefix}verificado",
        "user_followers_count": f"{prefix}seguidores",
        "user_following_count": f"{prefix}siguiendo",
        "user_tweet_count": f"{prefix}tweets",
        "user_listed_count": f"{prefix}listas"
    })
    .assign(us_fecha_creacion = lambda x: pd.to_datetime(x.us_fecha_creacion).dt.date)
)

users_renamed

In [ ]:
users.head()